In [30]:
#Importing so maybe VS code will let me debug the code
import ipykernel
ipykernel.__version__

'6.9.1'

In [31]:
import csv
import sys
import pandas as pd
import numpy as np

# Importing the files in different formats

In [61]:
from file_dir import video_dir, gaze_dir
points = ['world_index', 'gaze_normal0_x', 'gaze_normal0_y', 'gaze_normal0_z']

# The dataframe I plan on using
gaze_normal = pd.read_csv("C:\\Users\\deadg\\OneDrive\\Documents\\GithubRep\\SeniorCapstone\\recordings\\Temp\\004\\exports\\000\\gaze_positions.csv", usecols=points)

#A simplified table that consists of the x and y positions in the world image in normalized coordinates
#norm_pt_x = ['world_index', 'norm_pos_x']
#norm_pt_y = ['world_index', 'norm_pos_y']
#norm_pt_w = ['world_index']
#norm_pt = ['world_index', 'norm_pos_x', 'norm_pos_y']

#gaze_pos = pd.read_csv('C:/Users/deadg/OneDrive/Documents/GithubRep/SeniorCapstone/TextExtraction/ExportedData/OldTestData/gaze_positions.csv', usecols=points)

gaze_normal.head()

,world_index,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z
0,0,-0.228244,-0.086804,0.969727
1,1,-0.225766,-0.086061,0.970373
2,1,-0.224911,-0.082860,0.970850
3,2,-0.223600,-0.088945,0.970614
4,2,-0.227377,-0.086267,0.969978


Narrowing down data selection

In [54]:
#To complete step 2 we have to figure out the data looks like when the user is focused on an object
#The distance or size of an object is goind to present an issue with the way I have decided to select the frame
pd.set_option("display.max_rows", None, "display.max_columns", None)
gaze_normal.head()
pos_x = gaze_normal["gaze_normal0_x"]
pos_y = gaze_normal["gaze_normal0_y"]
pos_z = gaze_normal["gaze_normal0_z"]
w_index = gaze_normal["world_index"]
gaze_normal.head

<bound method NDFrame.head of      world_index  gaze_normal0_x  gaze_normal0_y  gaze_normal0_z
0              0       -0.228244       -0.086804        0.969727
1              1       -0.225766       -0.086061        0.970373
2              1       -0.224911       -0.082860        0.970850
3              2       -0.223600       -0.088945        0.970614
4              2       -0.227377       -0.086267        0.969978
5              3       -0.225378       -0.086673        0.970408
6              3       -0.226437       -0.086678        0.970162
7              3       -0.221285       -0.089697        0.971075
8              3       -0.222816       -0.090312        0.970668
9              3       -0.223611       -0.091622        0.970363
10             3       -0.217071       -0.091673        0.971842
11             4       -0.213443       -0.089293        0.972866
12             4       -0.217548       -0.090551        0.971840
13             4       -0.211532       -0.090380        0.97

Getting the index of the all the True values from tOrF and adding them to a list

In [59]:
#IT WORKS THIS IS IT BABY
# This takes out all the false values from the tOrF series
def frame_Extract(tOrF):
    list_Index = []
    count = 0 
    # Based on the size of the initial data, It runs through the list and gets the indexed number of each value that is equal to True and adds it to the list. 
    while count != tOrF.size:
        if tOrF.get(count) == True:
            #print("yes", y)
            list_Index.append(count)
        count += 1 #counter for the while loop
    
    # Splits the big list into separate list and adds them into a dictionary to be sorted
    # Still throws a index out of bounds execption tho in the
    gFrame = {}
    tempGroup = []
    count = 0
    groupNum = 0
    while count != len(list_Index) - 1:
        #if the next number is the expected number then the current number gets added to the list.
        if list_Index[count + 1] == list_Index[count] + 1:
            tempGroup.append(list_Index[count])
        else:
            # adds the last value to the current group before moving onto the next group of values.
            tempGroup.append(list_Index[count])
            gFrame["group{0}".format(groupNum)] = tempGroup
            tempGroup = []
            groupNum += 1
        count += 1

    # Time to clean up that data! Getting rid of the groups that have fewer then 10 items
    # Source: https://www.geeksforgeeks.org/python-ways-to-remove-a-key-from-dictionary/
    almostDone = {key: val for key, val in gFrame.items() if len(val) > 10}
    return match_Up(mid_part1(almostDone))

Finding the middle of each group of numbers

In [35]:
# Takes the cleaned Data and goes through each key value pair and sends the values to findMiddle
def mid_part1(almostDone):
    m_list = []
    for key in sorted(almostDone):
        m_list.append(mid_part2(almostDone[key]))
    return m_list

In [36]:
# Finds the middle element in the list and returns it.
# https://stackoverflow.com/questions/38130895/find-middle-of-a-list
def mid_part2(input_list):
    middle = float(len(input_list))/2
    if middle % 2 != 0:
        return input_list[int(middle - .5)]
    else:
        return input_list[int(middle)]

Getting the World_index for the Frames

In [37]:
# matching the selected coordinate indexes to its corresponding world_index 
def match_Up(mid_index):
    mid_Frame = []
    for x in mid_index:
        mid_Frame.append(w_index.get(x))
    return mid_Frame

In [38]:
def set_X_coords(lowest, highest):
    tOrF_X = pos_x.between(lowest, highest)
    return tOrF_X

In [39]:
def set_Y_coords(lowest, highest):
    tOrF_Y = pos_y.between(lowest, highest)
    return tOrF_Y

In [40]:
def set_Z_coords(lowest, highest):
    tOrF_Z = pos_z.between(lowest, highest)
    return tOrF_Z

# Main
Could do the same thing for the accuracy value too.
This program will take the X and Y coordinates from gaze_positions.csv and find the middle most frame of each grouped frame.

In [60]:
# Using the between method to make a true or false list based on a given range
#These are the values that we will need to change.
highest = .10
lowest = 0.0
# For the X coordinates
tOrF_X = set_X_coords(lowest, highest)# is a list of true or false. if the value in pos_x fits within that range it becomes true if not its false.
# For the Y coordinates
tOrF_Y = set_Y_coords(lowest, highest)

tOrF_Z = set_Z_coords(lowest, highest)

# Just double checking lengths
# Combined_TF is a combinded list of the tOrF list for the x, y and z coordinates.
if len(tOrF_X) == len(tOrF_Y) and len(tOrF_Z) == len(tOrF_X) and len(tOrF_Z) == len(tOrF_Y):
    Combined_TF = tOrF_X.eq(tOrF_Y)
    Combined_TF.eq(tOrF_Z)
else:
    print("YOU DONT WANT TO SEE THIS!!!!!!!!!!!!!!!!!!!!!!!!")
# Sending the data through the methods

mid_combined = frame_Extract(Combined_TF)
print("The Combined list of coordinates is ", mid_combined)
print("The chosen number is: ", mid_combined[0])
# Not sure if we still need this but it doesn't hurt
''' For some reason idk why this is having an issue 
mid_x = frame_Extract(tOrF_X)
print("The X coordinate Frames are ", mid_x)
mid_y = frame_Extract(tOrF_Y)
print("The Y coordinate Frames are ", mid_y)
mid_z = frame_Extract(tOrF_Z)
print("The Z coordinate Frames are ", mid_z)
'''

{'group0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105], 'group2': [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181], 'group5': [191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223], 'group6': [230, 23

' For some reason idk why this is having an issue \nmid_x = frame_Extract(tOrF_X)\nprint("The X coordinate Frames are ", mid_x)\nmid_y = frame_Extract(tOrF_Y)\nprint("The Y coordinate Frames are ", mid_y)\nmid_z = frame_Extract(tOrF_Z)\nprint("The Z coordinate Frames are ", mid_z)\n'